<a href="https://colab.research.google.com/github/chevamikado/ImageClassificationCNN/blob/master/TransferLearningMultipleCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Author: Muhammed Zahid Bozkus
## Lecture: INF003 - Deep Neural Networks

In [ ]:
## Import Libraries

import os
import datetime as dt
import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix

import pandas as pd
import numpy as np
import seaborn as sn
import itertools  

from PIL import Image
import xml.etree.ElementTree as ET
import psutil
import random

In [ ]:
## Specify sonme parameters

targetx = 224
targety = 224
learning_rate = 0.0001
classes = 120
seed = random.randint(1, 1000)

In [ ]:
## Install the Kaggle API

!pip install kaggle

In [ ]:
## Mount the drive to colab notebook

from google.colab import files

In [ ]:
## Upload your Kaggle API Token

files.upload()

In [ ]:
## Before importing the dataset we want to use this code
## The Kaggle API client expects this file to be in ~/.kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
## Download the dataset from Kaggle

!kaggle datasets download -d miljan/stanford-dogs-dataset-traintest

In [ ]:
## Unzip the dataset

local_zip = '/content/stanford-dogs-dataset-traintest.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/stanford-dogs')
zip_ref.close()

In [ ]:
## Paths for the train set and test set

train_data_dir = os.path.join("/content", "stanford-dogs", "cropped", "train")
test_data_dir = os.path.join("/content", "stanford-dogs", "cropped", "test")

In [ ]:
## Specify width-height for images and batch size

img_width, img_height = 128, 128
batch_size = 32

In [ ]:
## Generate batches of tensor image data with real-time data augmentation

train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        validation_split=0.5,
        preprocessing_function=preprocess_input
)

In [ ]:
## Generate bundle of augmented data. In the end we have a generator for the train, validation and test sets. 

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',    # 2D one-hot encoded labels (batch_size x 101)
       )
validation_generator = train_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',    # 2D one-hot encoded labels (batch_size x 101)
        subset='training')
test_generator = train_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_height, img_width),
        batch_size=1,
        class_mode='categorical',
        subset='validation')

In [ ]:
##  Specify callbacks

checkpoint = ModelCheckpoint('bestWeightsTL.h5',
                             monitor='val_accuracy',
                             save_best_only=True,
                             verbose=1,
                             mode='auto',
                             save_weights_only=False,
                             period=1)

earlystop = EarlyStopping(monitor='val_accuracy',
                          min_delta=.0001,
                          patience=20,
                          verbose=1,
                          mode='auto',
                          baseline=None,
                          restore_best_weights=True)

reducelr = ReduceLROnPlateau(monitor='val_accuracy',
                             factor=np.sqrt(.1),
                             patience=5,
                             verbose=1,
                             mode='auto',
                             min_delta=.0001,
                             cooldown=0,
                             min_lr=0.0000001)


In [ ]:
## Assign model to be used and add the output layers. 

base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(targetx, targety, 3))

## Uncomment following to assign another architecture
## base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(targetx, targety, 3))
## base_model = VGG16(include_top=False, weights='imagenet', input_shape=(targetx, targety, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
# x = Dropout(rate = .2)(x)
x = BatchNormalization()(x)
x = Dense(1280, activation='relu',  kernel_initializer=glorot_uniform(seed), bias_initializer='zeros')(x)
# x = Dropout(rate = .2)(x)
x = BatchNormalization()(x)
predictions = Dense(classes, activation='softmax', kernel_initializer='random_uniform', bias_initializer='zeros')(x)

model = Model(inputs=base_model.input, outputs=predictions)

## Specify optimizer 

optimizer = Adam(lr=learning_rate)

## Specify loss function
loss = "categorical_crossentropy"

## Compile the model

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy"])

## Summary of the model

model.summary()

In [ ]:
## Convert the Keras model to dot format and save to a file

tf.keras.utils.plot_model(
    base_model,
    to_file="model.png",
    show_shapes=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)

In [ ]:
## Start training the model

params = model.fit_generator(generator=train_generator, 
                                steps_per_epoch=len(train_generator), 
                                validation_data=validation_generator, 
                                validation_steps=len(validation_generator),
                                epochs=10,
                                callbacks=[earlystop, checkpoint, reducelr])

In [ ]:
## Load saved weights

model.load_weights('bestWeightsTL.h5')

In [ ]:
## Evaluate the model with the test set

loss, accuracy = model.evaluate(test_generator)

print("Test set loss: ", loss)
print("Test set accuracy: ", accuracy)

In [ ]:
## Plot accuracy for the train set and validation set

plt.plot(params.history['accuracy'])
plt.plot(params.history['val_accuracy'])
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', "Validation"], loc='upper right')
plt.show()

In [ ]:
## Plot loss for the train set and validation set

plt.plot(params.history['loss'])
plt.plot(params.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()